In [ ]:
import numpy as np
from numpy import array as arr
from numpy import zeros as zarr

import pandas as pd
from pandas import DataFrame as df
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
mpl.rcParams["font.serif"] = "CMU serif"
mpl.rcParams["mathtext.fontset"] = "custom"
mpl.rcParams["mathtext.rm"] = "CMU serif"
mpl.rcParams["mathtext.it"] = "CMU serif:italic"
mpl.rcParams["mathtext.bf"] = "CMU serif:bold"
mpl.rcParams["font.family"] = "serif"

# import pylandau
# from scipy import curve_fit
import scipy
import uncertainties as unc
from uncertainties import unumpy as unp
from uncertainties.unumpy import uarray as uarr
from uncertainties.unumpy import nominal_values as val
from uncertainties.unumpy import std_devs as dev
from uncertainties import ufloat as uf
# import ROOT

# import my plotting stuff
import sys
sys.path.append('./PythonHelpers/')
from PlotLib import Plotting
import PlotLib.Histogramming as Hist
import PlotLib.Plotting as Plot
from CSVimporter.runSettings import load_settings
from CSVimporter.importer import load_run
from CSVimporter.importer import dict_to_arr

import ROOT



def df_newcol(dframe, column, value, loc=-1):
    if column in dframe.columns and type(dframe.loc[0, column]) == type(value):
        print("WARNING: column", column, " already exists and has the same type")
        return
    if loc == -1:
        loc = len(dframe.columns)
    elif column in dframe.columns:
        loc = len(dframe.columns)
    try:
        try:
            dframe.insert(loc, column, pd.Series([value.copy()]*dframe.size, dtype=object))
        except:
            dframe.insert(loc, column, pd.Series([value.Clone()]*dframe.size, dtype=object))
    except:
        dframe.insert(loc, column, pd.Series([value]*dframe.size, dtype=object))
        
def df_query(frame, args, col, single=True):
    if type(args) == dict:
        query_str = " & ".join(["{:s}=={:n}".format(key, value) for key, value in args.items()])
    elif type(args) == list:
        query_str = " & ".join(["{:s}=={:n}".format(key, value) for key, value in args])
    elif type(args) == str:
        query_str = args
    index = frame.query(query_str).index
    if single:
        if len(index) != 1:
            raise Exception("Query returned {:n} entries".format(len(index)))
        index = index[0]
    return frame.at[index, col]
        
def get_hist_TH1(hHist):
    binN = hHist.GetNbinsX()
    bins = zarr(binN+1)    
    vals = zarr(binN)
    bins[0] = hHist.GetBinLowEdge(1)
    for i in range(binN):
        bins[i+1] = hHist.GetBinLowEdge(i+2)
        vals[i] = hHist.GetBinContent(i+1)
    return vals, bins


def truncate_hist(vals, bins, interval=0.95):
    vals_norm = vals / np.sum(vals)
    bin_centers = (bins[1:]+bins[:-1])/2
    Mean = np.sum(vals_norm*bin_centers)
    Stop = (1-interval)/2

    S = 0
    for i in range(len(vals)):
        S += vals_norm[i]
        if S > Stop:
            i_low = i
            break
    S = 1
    for i in range(len(vals)-1, -1, -1):
        S -= vals_norm[i]
        if S < 1-Stop:
            i_up = i
            break
    
    # need to include bin i_up, thus return up to i_up+1
    return vals[i_low:i_up+1], bins[i_low:i_up+2]

In [ ]:
thr = 220
runID = 190
SEC = 0.5
MTP = 6

FilePath = "/home/jona/DESY/analysis_TB/output/scan_tracking/"
# FileName = FilePath+str(runID) + "_thr{:n}".format(thr) + "_TrkCut{:n}".format(SCA_trk) + "_TrkPlanes{:n}".format(MTP) + "_AssocCut{:n}".format(SCA_assoc) + "_SEC{:.1f}".format(SEC) + ".root"
FileName = FilePath+str(runID) + "_thr{:n}".format(thr) + "_trkPlanes{:n}".format(MTP) + "_wideCuts" + ".root"
# output/scan_tracking/190_thr220_TrkCut150_TrkPlanes6_AssocCut60_SEC0.5.root

rFile = ROOT.TFile(FileName)
if not rFile:
    print("File not found")
    rFile.Close()

# Get the residual histograms
trunc_interval = 0.99
N_Planes = 6
residual_means = np.zeros([N_Planes,2]) # [ plane , x/y ]
residual_sigs = np.zeros([N_Planes,2]) # [ plane , x/y ]
# residual_mean = np.zeros([N_Planes,2], dtype=object) # 
# residual_sig = np.zeros([N_Planes,2], dtype=object)

for plane in range(N_Planes):
    nameDir = f"Tracking4D/adenium_{plane:n}/local_residuals"
    rDir = rFile.Get(nameDir)
    if not rDir:
        raise Exception("Dir.", nameDir, "not found in root file")
    else:
        # Get residual-x histogram & fit to get gaussian
        nameObj = "LocalResidualsX"
        hResidualX = rDir.Get(nameObj)
        if not hResidualX:
            raise Exception("Object", nameObj, "not found in root file at dir.", nameDir)
        else:
            vals_preTrunc, bins_preTrunc = get_hist_TH1(hResidualX)
            # print(residual_bins[plane,0])
            if True:
                xy = 0
                hist, bins = truncate_hist(vals_preTrunc, bins_preTrunc, interval=trunc_interval)
                bin_centers = (bins[1:]+bins[:-1])/2
                M = np.sum(hist*bin_centers)/np.sum(hist)
                RMS = np.sqrt(np.sum(hist*(bin_centers-M)**2)/np.sum(hist))
                residual_means[plane, xy] = M
                residual_sigs[plane, xy] = RMS
        # Get residual-y histogram & fit to get gaussian
        nameObj = "LocalResidualsY"
        hResidualY = rDir.Get(nameObj)
        if not hResidualY:
            raise Exception("Object", nameObj, "not found in root file at dir.", nameDir)
        else:
            vals_preTrunc, bins_preTrunc = get_hist_TH1(hResidualY)
            # print(residual_bins[plane,0])
            if True:
                xy = 1
                hist, bins = truncate_hist(vals_preTrunc, bins_preTrunc, interval=trunc_interval)
                bin_centers = (bins[1:]+bins[:-1])/2
                M = np.sum(hist*bin_centers)/np.sum(hist)
                RMS = np.sqrt(np.sum(hist*(bin_centers-M)**2)/np.sum(hist))
                residual_means[plane, xy] = M
                residual_sigs[plane, xy] = RMS
        
rFile.Close()

runSettings = load_settings(runID)
residual_sigs

In [ ]:
# draw residual sigma histogram
fig, ax = Hist.create_fig(1,1, figsize=(4,3))

# x
x = arr(range(N_Planes))
y = val(residual_sigs[:,0])
yerr = dev(residual_sigs[:,0])
ax.errorbar(x, y*1000, yerr=yerr*1000, fmt=".", label="x residual", color="black")

y = val(residual_sigs[:,1])
yerr = dev(residual_sigs[:,1])
ax.errorbar(x, y*1000, yerr=yerr*1000, fmt=".", label="y residual", color="darkred")


ax.set_xticks(range(N_Planes))
ax.set_xlim(-0.5, N_Planes-0.5)

Hist.finalize(runSettings, fig, ax, title=f"Tracking residuals\nstandard deviation", xlabel=r"adenium plane (along beam)", ylabel=r"$\sigma\,_{"+f" trunc. {trunc_interval*100:n}"+r"\%}$ / um", param_narrow=True, grid=False, legend_loc="lower right", thr=thr, ER1=False)

n_sigma = 5
cuts = max(residual_sigs[:,0])*n_sigma*1000, max(residual_sigs[:,1])*n_sigma*1000
print(f"cuts at {n_sigma} sigma of largest residuals")
print("x:", cuts[0])
print("y:", cuts[1])